In [1]:
#%pylab inline
#%config InlineBackend.figure_format = 'retina' 

In [2]:
#here are some common modules:
import scipy as sp #library of scientific functions
import scipy.io 
import scipy.signal as signal
import numpy as np #library of math functions
import pandas as pd #library of data analysis functions
import matplotlib.pyplot as plt #functions to plot data
import os #This lets python talk to your opperating system to open and save files.
from parabolic import parabolic
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import Axes3D

In [3]:
filename = 'emodat.mat' #adjut file name here
filename = os.path.join('../FM-BCI', filename) #adjust filepath 
datafile = sp.io.loadmat(filename) #loading filename
#print datafile.keys()
voltageSamples = datafile['data'] 
#print voltageSamples.shape, len(voltageSamples)

In [4]:
variances = []
means = []
medians = []
standardDeviations = []
for i in range(len(voltageSamples)):
    variances.append(np.var(voltageSamples[i,:]))
    means.append(np.mean(voltageSamples[i,:]))
    medians.append(np.median(voltageSamples[i,:]))
    standardDeviations.append(np.std(voltageSamples[i,:]))

In [5]:
for i in range(len(voltageSamples)):
    print 'Channel', i+1
    print '\tmean:', means[i]
    print '\tmedian:', medians[i]
    print '\tstandard deviation:', standardDeviations[i]
    print '\tvariance:', np.var(voltageSamples)

Channel 1
	mean: 53.5051604742
	median: 8.42684103561
	standard deviation: 105.317167517
	variance: 11091.7057739


In [6]:
fig1 = plt.figure(1, figsize=(9,6))
numBins = 50
n, bins, patches = plt.hist(voltageSamples[0],numBins,alpha=0.8)


In [7]:
fig2 = plt.figure(2, figsize=(9,6))
plt.plot(voltageSamples[0])


In [8]:
sampleRate = 512 # sample rate assumed during recording
sampleSpacing = 1.0 / sampleRate # time between samples in seconds
dataLengthSecs = 30 # length of whole recording
dataLengthSamples = dataLengthSecs*sampleRate # length of whole recording in samples

t = np.arange(0,dataLengthSecs,sampleSpacing) # time vector spanning length in seconds, with approp. num of samples
#numOfChannel = voltageSamples.shape[0] # determine number of channels from data, i.e. not predefined
numOfChannel = 8
voltageSamples = np.empty([numOfChannel, dataLengthSamples])

# channel weighting
channelWeights = np.linspace(1,1./numOfChannel,numOfChannel) 
#np.random.shuffle(channelWeights)
print 'Channel weights:', '\n'
for channelIndex in range(numOfChannel):
    print "        ", channelIndex+1, "   ", channelWeights[channelIndex]
    
# frequency modulation parameters
alphaCenter = 10.25   # Hz the carrier frequency
alphaModFreq = 1  # Hz the modulating frequency
alphaFreqDev = 4   # Hz of the frequency deviation

# signal to noise parameters
snr = 2             # signal / noise
noiseMean = 0
noiseStdDev = 0.5
alphaMean = 0
alphaStdDev = abs(np.sqrt(snr*(noiseStdDev**2))) # std of sine wave
alphaAmp = np.sqrt(2)*alphaStdDev
h = alphaFreqDev/alphaModFreq         # Modulation index

# Constructs 1/f noise by iteratively adding normal random noise, effectively the CDF of normal dist.
normalNoise = np.random.normal(noiseMean, noiseStdDev, (1,dataLengthSamples))
pinkNoise = np.cumsum(normalNoise)

# Frequency modulated alpha rhythm, a sinusoidal baseband signal
alpha = alphaAmp*np.sin( alphaCenter  * 2.0 * np.pi * t + alphaFreqDev*np.sin(2 * np.pi * alphaModFreq * t) / alphaModFreq)

# assign the weighted alpha rhythm + 1/f noise + additional random noise to each channel in sample
for channelIndex in range(0,numOfChannel):
    voltageSamples[channelIndex,:] = channelWeights[channelIndex]*alpha + pinkNoise + 0.5*np.random.random([1,dataLengthSamples])    
voltageSamples[channelIndex,:] = alpha # change last channel to ground truth alpha

fig3 = plt.figure(3, figsize=(9,6))

channelArray = np.vsplit(voltageSamples,1)
for channelIndex in range(0,numOfChannel):
    plt.plot(voltageSamples[channelIndex,1:sampleRate])



Channel weights: 

         1     1.0
         2     0.875
         3     0.75
         4     0.625
         5     0.5
         6     0.375
         7     0.25
         8     0.125


In [10]:
def butter_bandpass(lowcut, highcut, fs, order=4):
        #lowcut is the lower bound of the frequency that we want to isolate
        #hicut is the upper bound of the frequency that we want to isolate
        #fs is the sampling rate of our data
        nyq = 0.5 * fs #nyquist frequency - see http://www.dspguide.com/ if you want more info
        low = float(lowcut) / nyq
        high = float(highcut) / nyq
        b, a = sp.signal.butter(order, [low, high], btype='band')
        return b, a

def butter_bandpass_filter(mydata, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sp.signal.filtfilt(b, a, mydata)
    return y

#winLengthSecs = 1 # predefine length of window.
#winLengthSamples = winLengthSecs*sampleRate # length of window in samples
#numOfWindows = int(dataLengthSamples/winLengthSamples) # determine number of windows
#channelPeaks = np.empty([numOfChannel, numOfWindows]) # container for peak frequencies for each channel every second
desiredFreqResolution = 0.1 # predefine resolution of spectrum
#fftLengthSamples = int(sampleRate/desiredFreqResolution)
#nyq = 0.5*sampleRate # maximum possible frequency to measure
#freqs = scipy.fftpack.rfftfreq(fftLengthSamples,sampleSpacing) # retrieve frequency axis

bandLow = 9.5                                # lower alpha band 
bandHigh = 10.5                              # higher alpha band
orderFilter = 4  

# Universal FFT parameters
sampleRate = float(sampleRate)
numOfChannel = voltageSamples.shape[0]
dataLengthSamples = voltageSamples.shape[1]
dataLengthSecs = dataLengthSamples / sampleRate
sampleSpacing = 1.0/sampleRate        
nyq = 0.5 * sampleRate
fftLengthSamples = int(sampleRate/desiredFreqResolution)
freqs = np.fft.rfftfreq(fftLengthSamples,sampleSpacing)
winLengthSamples = 512
overlapSamples = 0
stepSize = winLengthSamples

medianSpecMat = np.empty([numOfChannel, len(freqs)])
for channelIndex in range(numOfChannel):
    winSpectra = np.empty(len(freqs))
    stepIndex = 0
    stepIndex = 0
    winStart = 0
    while winStart + winLengthSamples < dataLengthSamples:
        # get next window
        winStart = stepIndex*stepSize
        winStop = winStart + winLengthSamples
        voltageSamplesWin = voltageSamples[channelIndex,winStart:winStop]

        # detrend
        voltageSamplesWin = signal.detrend(voltageSamplesWin, axis=-1, type='linear')

        # window window
        windowedWin = voltageSamplesWin * signal.hanning(winLengthSamples)

        # compute fft
        amp = abs(np.fft.rfft(windowedWin,fftLengthSamples))
        winSpectra = np.c_[winSpectra,amp]
        stepIndex+=1
    winSpectra = winSpectra.T
    if channelIndex == 0:
        arrayOfAmps = winSpectra
        numOfWindows = stepIndex+1
    else:
        arrayOfAmps= np.dstack([arrayOfAmps, winSpectra])
arrayOfAmps = np.reshape(arrayOfAmps,(numOfChannel,numOfWindows,len(freqs)))
print medianSpecMat

[[  1.90519515e+02   2.54451559e+00   3.04509218e+00 ...,   7.04690573e-03
    7.09547633e-03   1.81753045e+01]
 [  5.86533484e-03   5.77486340e-03   5.86533484e-03 ...,   3.34732643e-04
    8.39316804e+00   3.07888056e-04]
 [  2.96399521e-04   3.07888055e-04   2.96399521e-04 ...,   2.28426808e+00
    7.40601474e-05   7.21721282e-05]
 ..., 
 [  1.23446634e-05   1.28733468e-05   1.23446634e-05 ...,   5.89897439e-06
    6.36154630e-06   5.89897488e-06]
 [  6.36154723e-06   5.89897386e-06   6.14425336e+00 ...,   2.78017789e-06
    2.57473135e-06   2.78017795e-06]
 [  2.57473137e-06   2.01339621e+00   2.91059910e-06 ...,   9.81260978e-07
    8.23069203e-07   9.81260804e-07]]


In [12]:
print np.median(medianSpecMat,1)
print np.mean(medianSpecMat,1)

[  2.58546795e+00   7.76256471e-04   1.07515938e-04   3.43089255e-05
   1.48576467e-05   7.28655671e-06   3.55765001e-06   1.30505190e-06]
[ 17.58047413   1.68692991   0.6946184    0.5022736    0.5018409
   0.49429549   0.42453923   0.42787643]


In [15]:
chPeakVariances = []
chPeakMeans = []
chPeakMedians = []
chPeakStandardDeviations = []

print numOfChannel

for i in range(len(medianSpecMat)):
    chPeakVariances.append(np.var(medianSpecMat[i,:]))
    chPeakMeans.append(np.mean(medianSpecMat[i,:]))
    chPeakMedians.append(np.median(medianSpecMat[i,:]))
    chPeakStandardDeviations.append(np.std(medianSpecMat[i,:]))

8


In [14]:
for i in range(len(medianSpecMat)):
    print 'Channel ' + str(i+1) + ' Peaks'
    print '\tmean:', chPeakMeans[i]
    print '\tmedian:', chPeakMedians[i]
    print '\tstandard deviation:', chPeakStandardDeviations[i]
    print '\tvariance:', chPeakVariances[i]

Channel 1 Peaks
	mean: 17.5804741313
	median: 2.58546794967
	standard deviation: 30.8914596152
	variance: 954.282277159
Channel 2 Peaks
	mean: 1.68692991212
	median: 0.000776256470593
	standard deviation: 5.06991975374
	variance: 25.7040863093
Channel 3 Peaks
	mean: 0.694618395097
	median: 0.000107515937776
	standard deviation: 2.13237167682
	variance: 4.54700896809
Channel 4 Peaks
	mean: 0.502273602069
	median: 3.43089254606e-05
	standard deviation: 1.54890059978
	variance: 2.39909306799
Channel 5 Peaks
	mean: 0.501840895959
	median: 1.48576466608e-05
	standard deviation: 1.46604382382
	variance: 2.14928449336
Channel 6 Peaks
	mean: 0.494295488674
	median: 7.28655670711e-06
	standard deviation: 1.51714166949
	variance: 2.3017188453
Channel 7 Peaks
	mean: 0.424539229266
	median: 3.55765001137e-06
	standard deviation: 1.25467539458
	variance: 1.57421034577
Channel 8 Peaks
	mean: 0.427876430277
	median: 1.3050519027e-06
	standard deviation: 1.29380592243
	variance: 1.67393376493


In [29]:
# Do this for all channels, one figure using subplot, each channel with its own subplot
fig4 = plt.figure(4, figsize=(9,9))
numBins = 50

for channelIndex in range(0, numOfChannel):
    print channelIndex
    plt.subplot(np.ceil(np.sqrt(numOfChannel)),np.ceil(np.sqrt(numOfChannel)),channelIndex)
    print channelIndex
    n, bins, patches = plt.hist(medianSpecMat[channelIndex],numBins,alpha=0.8)
    y = mlab.normpdf( bins, chPeakMeans[channelIndex],chPeakStandardDeviations[channelIndex] )
    l = plt.plot(bins, y, 'r--', linewidth=2)

for channelIndex in range(0, numOfChannel):
    n, bins, patches = plt.hist(medianSpecMat[7],numBins,alpha=0.8)
    #plt.show()

0


ValueError: num must be 1 <= num <= 9, not 0

In [28]:
# plot amp for each channel
#for each channel
    #for each window
        #plot 3d figure with time on horizontal, freqs on vertical, and winAmp as the fluctuating value
fig5 = plt.figure(5, figsize=(6,6))

#plt.plot(arrayOfAmps[0,1,:])
#plt.show()
#print arrayOfAmps[0][0]
#ax = Axes3D(fig5)
ax = plt.axes(projection='3d')
#for channelIndex in range(0, numOfChannel):
#print np.shape(arrayOfAmps[0])
x = range(0,numOfWindows)
y = freqs
X,Y = np.meshgrid(x,y)
X,Y = X.T,Y.T
print np.shape(X)
print np.shape(Y)
Z = arrayOfAmps[0]
print Z
Z = Z.reshape(Y.shape)
ax.plot_surface(X,Y,Z)
#plt.ylim(0,12)
plt.show()
#for winIndex in range(0, numOfWindows):
    
    #for lengthOfAmps in range(0, len(freqs)):
         #   plt.subplot(ceil(sqrt(numOfChannel)),ceil(sqrt(numOfChannel)),lengthOfAmps)
#        ax.plot_surface( arrayOfAmps[0], arrayOfAmps[0][winIndex], arrayOfAmps[0][winIndex][lengthOfAmps] ,  cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)    

#for zIndex in range(0, len(freqs)):
#   ax.plot(arrayOfAmps[channelIndex][winIndex], arrayOfAmps[channelIndex], arrayOfAmps[channelIndex][winIndex][zIndex])

(31, 2561)
(31, 2561)
[[  0.00000000e+00   1.93015606e+02   1.94256514e+02 ...,   1.40582256e+01
    1.82093528e+01   6.40000000e+00]
 [  1.59139628e+01   1.56683388e+01   1.78025210e+01 ...,   9.39808571e+00
    3.84000000e+01   9.16892314e+00]
 [  9.03026808e+00   8.85456178e+00   1.06695956e+01 ...,   7.04000000e+01
    3.29341503e+00   3.84176302e+00]
 ..., 
 [  1.71505316e+00   3.57403604e+00   4.81977645e+00 ...,   7.48542664e+00
    2.82316144e+00   4.85640794e+00]
 [  5.55939190e+00   6.15599129e+00   1.23446635e-05 ...,   8.57533799e+00
    5.39325291e+00   5.38571898e+00]
 [  5.92367337e+00   5.89897439e-06   3.68738771e+00 ...,   1.52508229e+00
    2.88847939e+00   2.64135269e+00]]
